In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Lung_Cancer"
cohort = "GSE21359"

# Input paths
in_trait_dir = "../../input/GEO/Lung_Cancer"
in_cohort_dir = "../../input/GEO/Lung_Cancer/GSE21359"

# Output paths
out_data_file = "../../output/preprocess/Lung_Cancer/GSE21359.csv"
out_gene_data_file = "../../output/preprocess/Lung_Cancer/gene_data/GSE21359.csv"
out_clinical_data_file = "../../output/preprocess/Lung_Cancer/clinical_data/GSE21359.csv"
json_path = "../../output/preprocess/Lung_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Association of CXCL14 in the Human Airway Epithelium with Chronic Obstructive Lung Disease and Lung Cancer"
!Series_summary	"CXCL14, a recently described chemokine constitutively expressed in various epithelia, has multiple putative roles in inflammation and carcinogenesis. Based on the knowledge that cigarette smoking and the smoking-induced disorders, such as chronic obstructive pulmonary disease (COPD) and lung cancer, are associated with inflammation, we hypothesized that the airway epithelium, the primary site of smoking-induced pathologic changes in COPD and adenocarcinoma, responds to cigarette smoking with an altered CXCL14 gene expression as a part of disease-relevant molecular phenotype. Microarray analysis with subsequent TaqMan PCR validation revealed very low constitutive CXCL14 gene expression in the airway epithelium of healthy nonsmokers (n=53) which was strongly up-regulated in healthy smokers ( n=59; p<0.001) and further increase

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset appears to contain gene expression data from Affymetrix arrays
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Data Availability and 2.2 Data Type Conversion

# Trait (Lung Cancer vs. COPD vs. Normal)
trait_row = 3  # The smoking status row contains disease information

def convert_trait(value):
    if not value or ':' not in value:
        return None
    
    value = value.split(':', 1)[1].strip().lower()
    
    # This dataset is about COPD, healthy smokers, and non-smokers
    if 'copd' in value:
        return 1  # COPD patients
    elif 'smoker' in value and 'non-smoker' not in value:
        return 0  # Healthy smokers
    elif 'non-smoker' in value:
        return 0  # Non-smokers are also controls
    else:
        return None

# Age
age_row = 0  # Age information is in row 0

def convert_age(value):
    if not value or ':' not in value:
        return None
    
    try:
        # Extract the numeric value after the colon
        age_value = value.split(':', 1)[1].strip()
        return float(age_value)  # Convert to float for continuous variable
    except:
        return None

# Gender
gender_row = 1  # Gender information is in row 1

def convert_gender(value):
    if not value or ':' not in value:
        return None
    
    gender = value.split(':', 1)[1].strip().upper()
    
    if gender == 'M':
        return 1  # Male
    elif gender == 'F':
        return 0  # Female
    else:
        return None

# 3. Save Metadata
# Initial filtering on the usability of the dataset
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=trait_row is not None
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we extract clinical features
if trait_row is not None:
    # Load the clinical data from the previous step (assuming it's available as clinical_data)
    # Extract clinical features
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    print("Preview of extracted clinical features:")
    print(preview_df(clinical_features))
    
    # Save the clinical features to a CSV file
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of extracted clinical features:
{'GSM101095': [0.0, 41.0, 1.0], 'GSM101096': [0.0, 35.0, 1.0], 'GSM101097': [0.0, 61.0, 1.0], 'GSM101098': [0.0, 37.0, 0.0], 'GSM101100': [0.0, 47.0, 1.0], 'GSM101101': [0.0, 38.0, 1.0], 'GSM101102': [0.0, 49.0, 0.0], 'GSM101103': [0.0, 45.0, 1.0], 'GSM101104': [0.0, 36.0, 1.0], 'GSM101105': [0.0, 38.0, 1.0], 'GSM101106': [0.0, 35.0, 1.0], 'GSM101107': [0.0, 46.0, 1.0], 'GSM101111': [0.0, 37.0, 0.0], 'GSM101113': [0.0, 45.0, 1.0], 'GSM101114': [0.0, 48.0, 1.0], 'GSM101115': [0.0, 50.0, 1.0], 'GSM101116': [0.0, 46.0, 0.0], 'GSM114089': [0.0, 56.0, 1.0], 'GSM114090': [0.0, 59.0, 1.0], 'GSM190149': [0.0, 49.0, 1.0], 'GSM190150': [0.0, 34.0, 1.0], 'GSM190151': [0.0, 44.0, 1.0], 'GSM190152': [0.0, 45.0, 1.0], 'GSM190153': [0.0, 45.0, 0.0], 'GSM190154': [0.0, 29.0, 0.0], 'GSM190155': [0.0, 42.0, 1.0], 'GSM190156': [0.0, 56.0, 1.0], 'GSM252828': [1.0, 47.0, 1.0], 'GSM252829': [1.0, 47.0, 1.0], 'GSM252830': [1.0, 50.0, 1.0], 'GSM252831': [1.0, 55.0, 1.0]

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Review the gene identifiers from the previous output
# These identifiers appear to be Affymetrix probe IDs (e.g., '1007_s_at', '1053_at')
# rather than standard human gene symbols like BRCA1, TP53, etc.
# Affymetrix probe IDs need to be mapped to gene symbols for biological interpretation

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns for gene identifiers and gene symbols in the gene annotation
# Based on the preview, 'ID' contains probe identifiers and 'Gene Symbol' contains gene symbols
prob_col = 'ID'
gene_col = 'Gene Symbol'

# 2. Get gene mapping dataframe by extracting the two columns
gene_mapping = get_gene_mapping(gene_annotation, prob_col, gene_col)

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
# This divides probe values equally among mapped genes and sums all probe values for each gene
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Preview the first few genes after mapping
print("Preview of gene expression data after mapping:")
print(gene_data.index[:20])


Preview of gene expression data after mapping:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1',
       'A4GALT', 'A4GNT', 'AA06', 'AAAS', 'AACS', 'AACSP1', 'AADAC', 'AADACL2',
       'AADACP1', 'AADAT', 'AAED1', 'AAGAB', 'AAK1'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Shape of normalized gene data: {normalized_gene_data.shape}")
# Save to the gene data file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Saved normalized gene data to {out_gene_data_file}")

# 2. Link the clinical and genetic data
# Need to load the saved clinical data
clinical_features = pd.read_csv(out_clinical_data_file)

# Print clinical_features structure to debug
print(f"Clinical features columns: {clinical_features.columns.tolist()}")
print(f"Clinical features shape: {clinical_features.shape}")

# Convert clinical features to the correct format for linking
# First, we need to set the appropriate index
if 'Unnamed: 0' in clinical_features.columns:
    clinical_features = clinical_features.set_index('Unnamed: 0')

# Create a new DataFrame with the trait name as Lung_Cancer for clarity
trait_row = clinical_features.iloc[0].rename(trait)
age_row = clinical_features.iloc[1].rename('Age') if len(clinical_features) > 1 else None
gender_row = clinical_features.iloc[2].rename('Gender') if len(clinical_features) > 2 else None

# Combine rows into a new clinical dataframe with proper naming
clinical_df_rows = [trait_row]
if age_row is not None:
    clinical_df_rows.append(age_row)
if gender_row is not None:
    clinical_df_rows.append(gender_row)

named_clinical_df = pd.DataFrame(clinical_df_rows)
print(f"Named clinical dataframe shape: {named_clinical_df.shape}")
print(f"Named clinical dataframe index: {named_clinical_df.index.tolist()}")

# Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(named_clinical_df, normalized_gene_data)
print(f"Shape of linked data: {linked_data.shape}")
print(f"First few columns in linked_data: {linked_data.columns[:10].tolist()}")

# Check if the trait column exists in the dataframe
if trait not in linked_data.columns:
    print(f"Warning: '{trait}' column not found in linked data. Available columns: {linked_data.columns[:20].tolist()}")

# 3. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Shape of linked data after handling missing values: {linked_data.shape}")

# 4. Determine whether the trait and demographic features are biased, and remove biased features
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Dataset contains gene expression from olfactory neuroblastoma patients relevant to lung cancer research"
)

# 6. If the linked data is usable, save it as a CSV file
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Saved processed linked data to {out_data_file}")
else:
    print("Dataset validation failed. Data not saved.")

Shape of normalized gene data: (19845, 135)


Saved normalized gene data to ../../output/preprocess/Lung_Cancer/gene_data/GSE21359.csv
Clinical features columns: ['Unnamed: 0', 'GSM101095', 'GSM101096', 'GSM101097', 'GSM101098', 'GSM101100', 'GSM101101', 'GSM101102', 'GSM101103', 'GSM101104', 'GSM101105', 'GSM101106', 'GSM101107', 'GSM101111', 'GSM101113', 'GSM101114', 'GSM101115', 'GSM101116', 'GSM114089', 'GSM114090', 'GSM190149', 'GSM190150', 'GSM190151', 'GSM190152', 'GSM190153', 'GSM190154', 'GSM190155', 'GSM190156', 'GSM252828', 'GSM252829', 'GSM252830', 'GSM252831', 'GSM252833', 'GSM252835', 'GSM252836', 'GSM252837', 'GSM252838', 'GSM252839', 'GSM252841', 'GSM252871', 'GSM252876', 'GSM252879', 'GSM252880', 'GSM252881', 'GSM252882', 'GSM252884', 'GSM252885', 'GSM254149', 'GSM254150', 'GSM254151', 'GSM254152', 'GSM254157', 'GSM254158', 'GSM254159', 'GSM254160', 'GSM254161', 'GSM254163', 'GSM254169', 'GSM254172', 'GSM254173', 'GSM254174', 'GSM254175', 'GSM254176', 'GSM298219', 'GSM298220', 'GSM298221', 'GSM298222', 'GSM298223'

Shape of linked data after handling missing values: (135, 19848)
For the feature 'Lung_Cancer', the least common label is '1.0' with 23 occurrences. This represents 17.04% of the dataset.
The distribution of the feature 'Lung_Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 38.0
  50% (Median): 45.0
  75%: 48.0
Min: 21.0
Max: 73.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 40 occurrences. This represents 29.63% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

A new JSON file was created at: ../../output/preprocess/Lung_Cancer/cohort_info.json


Saved processed linked data to ../../output/preprocess/Lung_Cancer/GSE21359.csv
